In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs
import math

from nnfs.datasets import spiral_data
from nnfs.datasets import vertical_data

from layers import Dense
from activations import ReLU
from activations import SoftMax
from losses import CategoricalCrossEntropy, Softmax_CategoricalCrossentropy
from optimizers import SGD, AdaGrad, RMSProp, Adam

In [3]:
nnfs.init()

## CH 11: Testing with Out-of-Sample Data

In [5]:
# Dataset
X, y = spiral_data(samples=100, classes=3)

# First Layer
dense1 = Dense(2, 64)
activation1 = ReLU()

# Second Layer
dense2 = Dense(64, 3)

# Categorical-CrossEntropy with Activation
loss_activation = Softmax_CategoricalCrossentropy()

# Optimizer
optimizer = Adam(decay=5e-7, learning_rate=0.05)

for epoch in range(10001):
    # Forward Pass
    dense1.forward(X)
    activation1.forward(dense1.output)

    dense2.forward(activation1.output)
    # Loss Computation
    loss = loss_activation.forward(dense2.output, y)

    # Accuracy
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) ==2:
        y = np.argmax(y, axis=1)

    acc = np.mean(predictions==y)
    
    if not epoch % 1000:
        print(f'epoch: {epoch}, ' +
              f'acc: {acc:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward Pass
    loss_activation.backward(loss_activation.output, y)

    dense2.backward(loss_activation.dinputs)

    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Optimize
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.343, loss: 1.099, lr: 0.05
epoch: 1000, acc: 0.863, loss: 0.339, lr: 0.049975037468784345
epoch: 2000, acc: 0.867, loss: 0.289, lr: 0.04995007490013731
epoch: 3000, acc: 0.867, loss: 0.283, lr: 0.049925137256683606
epoch: 4000, acc: 0.903, loss: 0.236, lr: 0.049900224501110035
epoch: 5000, acc: 0.883, loss: 0.239, lr: 0.04987533659617785
epoch: 6000, acc: 0.897, loss: 0.227, lr: 0.04985047350472258
epoch: 7000, acc: 0.900, loss: 0.216, lr: 0.04982563518965381
epoch: 8000, acc: 0.907, loss: 0.207, lr: 0.04980082161395499
epoch: 9000, acc: 0.930, loss: 0.196, lr: 0.04977603274068329
epoch: 10000, acc: 0.923, loss: 0.188, lr: 0.04975126853296942


In [8]:
# Create test set
X_test, y_test = spiral_data(samples=100, classes=3)

# Evaluate the model on test set
# Forward pass

dense1.forward(X_test)
activation1.forward(dense1.output)

dense2.forward(activation1.output)

loss = loss_activation.forward(dense2.output, y_test)

predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) ==2:
    y_test = np.argmax(y_test, axis=1)

acc = np.mean(predictions==y_test)

print(f'Validation acc: {acc:.3f}, loss: {loss:.3f}')

Validation acc: 0.780, loss: 0.825
